# Exam simulation - ExamExample2

## Part 1 - MCQ

1. The true answer is: 'After applying “oneHotEncoderEstimator”, an integer is transformed into a vector', indeed that's how oneHotEncoderEstimator works, by transforming a column of tokens into a column of one-hot vectors.
2. The false option is: '"e2" is a graphframe', indeed it is a DataFrame (containing the edges of graph "g2"), as it is obtained by filtering "e".

## Part 2 - Exercises

### 1 

Isolate the cities with at least one day with max temperature > 35° and the with at least one day having min temperature < -20.

In [3]:
import sys

# open file
# in_file_1 = sys.argv[2]
in_file_1 = '/data/students/bigdata_internet/exam_examples_data/example2_data/Temperatures.txt'

# out_file_1 = sys.argv[1]
out_file_1 = '/user/s315054/exam_sim/example2/p01'

in_DF = spark.read.load(in_file_1, format='csv', sep=',', header=False, inferSchema=True)\
        .withColumnRenamed('_c0', 'date').withColumnRenamed('_c1', 'city')\
        .withColumnRenamed('_c2', 'country').withColumnRenamed('_c3', 't_max')\
        .withColumnRenamed('_c4', 't_min')

spark.udf.register('isMin', lambda t: int(t<-20))
spark.udf.register('isMax', lambda t: int(t>35))

ok_DF = in_DF.filter("t_max > 35 OR t_min < -20").selectExpr('city', 'date', 'isMin(t_min) AS min_ind', 'isMax(t_max) AS max_ind')

final_DF = ok_DF.groupBy('city').agg({'min_ind':'sum', 'max_ind':'sum'}).filter('sum(min_ind) > 0 AND sum(max_ind) > 0')

final_DF.select('city').write.csv(out_file_1, header=False)

23/01/25 10:36:56 WARN analysis.SimpleFunctionRegistry: The function ismin replaced a previously registered function.
23/01/25 10:36:56 WARN analysis.SimpleFunctionRegistry: The function ismax replaced a previously registered function.


### 2 Analyzing 2015 data

In [10]:
import sys

# in_file_2 = sys.argv[1]
# out_file_2a = sys.argv[2]
# out_file_2b = sys.argv[3]

in_file_2 = in_file_1

out_file_2a = '/user/s315054/exam_sim/example2/p02a'
out_file_2b = '/user/s315054/exam_sim/example2/p02b'

# in_DF = ...


#### a - average of the max temp. for each city in 2015

In [11]:
spark.udf.register('getYear', lambda date: date.split('/')[0])
spark.udf.register('concatenate', lambda c1, c2: str(c1)+'-'+str(c2))

in_DF.filter('getYear(date) == "2015"').groupBy('city', 'country').agg({'t_max':'avg'}).withColumnRenamed('avg(t_max)', 't_max_avg')\
        .selectExpr('concatenate(city, country) AS city_country', 't_max_avg')\
        .write.csv(out_file_2a, header=False)

23/01/25 10:50:28 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.
23/01/25 10:50:28 WARN analysis.SimpleFunctionRegistry: The function concatenate replaced a previously registered function.


#### b - hot cities in 2015

Hot cities are the ones for which the average max_temperature is at least 5°C greater than the average temperature in the country.

In [20]:
city_avg_DF = in_DF.filter('getYear(date) == "2015"').groupBy('city', 'country')\
                .agg({'t_max':'avg'}).withColumnRenamed('avg(t_max)', 't_max_avg')

city_avg_DF.createOrReplaceTempView('avg_cities')

country_avg_DF = in_DF.filter('getYear(date) == "2015"').groupBy('country')\
                    .agg({'t_max':'avg'}).withColumnRenamed('avg(t_max)', 't_max_avg')

country_avg_DF.createOrReplaceTempView('avg_countries')

final_DF = spark.sql("""
                    SELECT avg_cities.city, avg_cities.t_max_avg, avg_countries.t_max_avg
                    FROM avg_cities, avg_countries
                    WHERE avg_cities.country == avg_countries.country AND avg_cities.t_max_avg >= (avg_countries.t_max_avg + 5)
                    """)

In [15]:
city_avg_DF.show(20)

+-----+-------+------------------+
| city|country|         t_max_avg|
+-----+-------+------------------+
|Paris| France|21.166666666666668|
| Lyon| France|28.166666666666668|
| Rome|  Italy|              19.5|
|Turin|  Italy|             24.75|
+-----+-------+------------------+



In [17]:
country_avg_DF.show(20)

+-------+------------------+
|country|         t_max_avg|
+-------+------------------+
| France|24.666666666666668|
|  Italy|            22.125|
+-------+------------------+



In [21]:
final_DF.show()

+----+---------+---------+
|city|t_max_avg|t_max_avg|
+----+---------+---------+
+----+---------+---------+



In [ ]:
# IT WORKS

# With RDDS

It may be convenient, especially for the 1st exercise

In [45]:
# Re doing ex 01

in_file_1 = '/data/students/bigdata_internet/exam_examples_data/example2_data/Temperatures.txt'
out_file_1_rdd = '/user/s315054/exam_sim/example2/p01_rdd'

in_RDD = sc.textFile(in_file_1).map(lambda line: (line.split(',')[1], [float(n) for n in line.split(',')[3:]]))

out_RDD = in_RDD.reduceByKey(lambda c1, c2: [max(c1[0], c2[0]), min(c1[1], c2[1])])\
                .filter(lambda el: el[1][0] > 35 and el[1][1] < -20).keys()
out_RDD.saveAsTextFile(out_file_1_rdd)

In [44]:
out_RDD.collect()

['Turin']

In [42]:
in_RDD.collect()

[('Turin', [32.5, 26.0]),
 ('Turin', [30.5, 25.0]),
 ('Turin', [34.5, 24.0]),
 ('Turin', [35.5, 22.0]),
 ('Turin', [36.5, 21.0]),
 ('Turin', [32.5, 29.0]),
 ('Turin', [-0.5, -21.4]),
 ('Turin', [-3.5, -10.5]),
 ('Rome', [26.0, 19.5]),
 ('Rome', [25.0, 22.5]),
 ('Rome', [19.0, 19.5]),
 ('Rome', [28.0, 22.5]),
 ('Rome', [23.0, 20.0]),
 ('Rome', [15.5, 14.5]),
 ('Rome', [12.5, 10.0]),
 ('Rome', [7.0, 5.0]),
 ('Paris', [18.0, 10.0]),
 ('Paris', [22.0, 12.5]),
 ('Paris', [27.0, 14.5]),
 ('Paris', [31.0, 12.5]),
 ('Paris', [19.0, 10.5]),
 ('Paris', [10.0, 8.5]),
 ('Lyon', [31.0, 12.5]),
 ('Lyon', [29.0, 15.5]),
 ('Lyon', [27.0, 24.5]),
 ('Lyon', [34.0, 22.5]),
 ('Lyon', [28.0, 15.5]),
 ('Lyon', [20.0, 10.5])]